In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/V1.4_Training.csv')
print(train_df.head)
train_df.dropna(inplace=True)
print(train_df.shape)
sugg_samples = []
print(train_df.size)
for i in range(len(train_df)):
    if(train_df.iloc[i]['Tag']==1):
        sugg_samples.append(train_df.loc[i]['Sentence'])

In [ ]:
num_of_topic_words = 100
num_of_ngrams = 15000

## Extraction of all n-grams for n=5

In [ ]:
len(sugg_samples)

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import string
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english')) 

In [ ]:
textp = ' '.join(sugg_samples)
textp = textp.lower()
textp = textp.translate(textp.maketrans('','', string.punctuation))
textp = word_tokenize(textp)

In [ ]:
ngrams = lambda a, n: zip(*[a[i:] for i in range(n)])

In [ ]:
from collections import Counter
mc_sugg = Counter(ngrams(textp, 5)).most_common(num_of_ngrams)
mc_sugg

## Topic Modeling

In [ ]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk.stem as stemmer
import numpy as np
import collections

In [ ]:
np.random.seed(2018)
import nltk

In [ ]:
nltk.download('wordnet')

In [ ]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer('english')
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token!="https":
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
def string2cnov(text,dict):
            
    val=0
    pres=""
    flag=False
    for curr in range(0,len(text)-1):
        
        #print (text[curr] +" ")
        #print(pres+" ")
        if text[curr]=='*':
          val=float(pres)
          pres=""
        elif text[curr]=='"' and flag==False :
          
          flag=True
          
          curr=curr+1
          ster=""
          while (text[curr]!='"') :
                ster=ster+text[curr]
                curr=curr+1
          curr=curr+3
          filler=0
          if (ster!="."):
              if ster in dict.keys():
                  filler=dict[ster]
              dict[ster]=filler+val
              if (curr>=len(text)):
                break
          #print(text[curr])
          
          pres=""
          val=0
        elif text[curr]=='"' and flag==True :
            flag=False
        elif text[curr]=='.' or (text[curr]>='0' and text[curr]<='9'):
            pres=pres+text[curr]
    return dict

In [ ]:
tdictr = {}

In [ ]:
processed_docs = [preprocess(text) for text in sugg_samples]
# processed_docs[:10]

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    # print(k, v)
    count += 1
    if count > 50:
        break


In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[len(bow_corpus)-1]

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=100, id2word=dictionary, passes=2, workers=2)

In [ ]:
dictr={}
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    d=string2cnov(topic,dictr)
    t=string2cnov(topic,tdictr)

In [ ]:
newd = dict(Counter(dictr).most_common(num_of_topic_words))
values = list(newd.values())

In [ ]:
topic_words = set(newd.keys())

In [ ]:
topic_words

## Extraction of N-Grams with atleast one word among the most common keywords

In [ ]:
def ngrams_with_topics(mc_sugg, topic_words):
    filtered = []
    for ng in mc_sugg:
        flag = 0
        for word in ng[0]:
            if word in topic_words:
                flag = 1
        if flag:
            filtered.append(ng)
    return filtered

In [ ]:
filtered_mc_sugg = ngrams_with_topics(mc_sugg, topic_words)
filtered_mc_sugg

In [ ]:
filtered_mc_sugg = filtered_mc_sugg[:4330]

## Preprocessing for training language models

In [ ]:
def preprocess(df):
    inp = df.values
    for i in range(len(inp)):
        s = inp[i]
        sl = s.lower()
        sl = result = sl.translate(sl.maketrans('','', string.punctuation))
        tokens = word_tokenize(sl)
        #tokens = [w for w in tokens if not w in stop_words]
        inp[i] = tokens
    return inp

train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/V1.4_Training.csv')
train_df.dropna(inplace=True)
train_df = train_df[train_df['Tag']==1]['Sentence']
textall = preprocess(train_df)

In [ ]:
import numpy as np
en_model = {}
f = open('/content/drive/My Drive/data/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    en_model[word] = coefs
f.close()

print('Found %s word vectors.' % len(en_model))

In [ ]:
for t in textall:
    for w in t:
        if w in string.punctuation:
            t.remove(w)

In [ ]:
words=[]
keys=[]
i=1
print('starting')
for k, tsent in enumerate(textall):
    if (k%1000 == 0):
        print ('k', k, k/textall.size)
    for j, word in enumerate(tsent):
        #if (j%100 == 0):
            #print ('j', j)
        if word not in words:
            words.append(word)
            keys.append(i)
            i=i+1
print ('done!')

In [ ]:
embed_len = len(words)
print (embed_len)

word2id = dict(zip(words, keys))
id2word = dict(zip(keys, words))
embeddings = np.zeros((embed_len+1,len(en_model['test'])))  # num_words * 100 (word vec len)
print('starting')
for i, w in enumerate(words):
    try:
        vec = en_model[w]
    except KeyError:
        pass
    embeddings[word2id[w]]=vec
print('done')

In [ ]:
def ngrams_lang_inp(arr):
    inp = []
    curr = []
    for s in arr:
        i=0
        while(i+5<len(s)):
            ngram = list(s[i:i+6])
            idngram = [word2id[w] for w in ngram]
            if i+6<len(s):
                nextword = word2id[s[i+6]]   
            else:
                break
            curr = [idngram, nextword]
            inp.append(curr)
            curr = []
            i=i+1
        
    return inp

In [ ]:
sugg_text = []
sugg_len = len(sugg_samples)
print (sugg_len)
print ('starting')
for i in range(sugg_len):
    s = sugg_samples[i]
    sl = s.lower()
    sl = result = sl.translate(sl.maketrans('','', string.punctuation))
    tokens = word_tokenize(sl)
    sugg_text.append(tokens)
print ('finishing')
sugg_lang_seq = ngrams_lang_inp(sugg_text)
train_xp = np.array([s[0] for s in sugg_lang_seq])
train_yp = np.array([s[1] for s in sugg_lang_seq])

print('done')

## Training Language Models

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Embedding, Dense, Input, Bidirectional
from tensorflow.keras.models import Sequential

modelp = Sequential()
modelp.add(Embedding(embeddings.shape[0], output_dim = 100,
                     weights = [embeddings], input_length=6, trainable = True))
modelp.add(Bidirectional(LSTM(256, activation = 'relu')))
modelp.add(Dense(1024, activation = 'relu'))
modelp.add(Dense(embeddings.shape[0], activation = 'softmax'))

modelp.compile(optimizer = 'Adam',
               loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
modelp.fit(train_xp, train_yp, batch_size = 64, epochs=20, verbose=True)

In [ ]:
modelp.save('/content/drive/My Drive/data/sugg_model6')

## Loading Language Models

In [ ]:
import keras
model4 = keras.models.load_model('/content/drive/My Drive/data/sugg_model4')
model5 = keras.models.load_model('/content/drive/My Drive/data/sugg_model5')
model6 = keras.models.load_model('/content/drive/My Drive/data/sugg_model6')

## Generation of synthetic datapoints using the three language models

In [ ]:
#Randomized output of 3 language models for generation of synthetic sample
import numpy as np
sugg_generated = []
for sug in filtered_mc_sugg:
    ngram_gen = sug[0] #('', '', '', '', '')
    idngram = []
    for p in ngram_gen:
        idngram.append(word2id[p])
    #[1, 2, 3, 4, 5]
    ran_len = np.random.randint(20, 30)
    sentence = [id2word[i] for i in idngram]
    curr5 = idngram[:5]
    curr4 = idngram[1:5]
    curr6 = idngram[:5]
    for l in range(ran_len):
        inp4 = np.array([curr4, [2, 3, 4, 5]])
        out4 = model4.predict(inp4)
        i4 = np.argmax(out4[0])
        max4 = out4[0][i4]
        inp5 = np.array([curr5, [2, 3, 4, 5, 6]])
        out5 = model5.predict(inp5)
        i5 = np.argmax(out5[0])
        max5 = out5[0][i5]
        if l != 0:
          inp6 = np.array([curr6, [2, 3, 4, 5, 6, 7]])
          out6 = model6.predict(inp6)
          i6 = np.argmax(out6[0])
          max6 = out6[0][i6]

        if l != 0:
          z = np.random.randint(0, 2)

        else:
          z = np.random.randint(0, 1)

        if z==0:
          p = i4
          wordp = id2word[p]
          sentence.append(wordp)
          curr4 = np.delete(curr4, 0)
          curr4 = np.append(curr4, p)
          curr5 = np.delete(curr5, 0)
          curr5 = np.append(curr5, p)
          if l != 0:
            curr6 = np.delete(curr6, 0)
            curr6 = np.append(curr6, p)

          else:
            curr6 = np.append(curr6, p)

        elif z==1:
          p = i5
          wordp = id2word[p]
          sentence.append(wordp)
          curr4 = np.delete(curr4, 0)
          curr4 = np.append(curr4, p)
          curr5 = np.delete(curr5, 0)
          curr5 = np.append(curr5, p)
          if l != 0:
            curr6 = np.delete(curr6, 0)
            curr6 = np.append(curr6, p)

          else:
            curr6 = np.append(curr6, p)

        elif z==2:
          p = i6
          wordp = id2word[p]
          sentence.append(wordp)
          curr4 = np.delete(curr4, 0)
          curr4 = np.append(curr4, p)
          curr5 = np.delete(curr5, 0)
          curr5 = np.append(curr5, p)
          curr6 = np.delete(curr6, 0)
          curr6 = np.append(curr6, p)
    sugg_generated.append(sentence)

In [ ]:
final_list = []
for l in sugg_generated:
  sentence = " ".join(l)
  final_list.append(sentence)

In [ ]:
final_list

## Augmenting the dataset with the synthetic samples

In [ ]:
import pandas as pd
df = pd.DataFrame(final_list, columns=["Sentence"])
df['Tag'] = 1

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/V1.4_Training.csv')
result = train_df.append(df)

In [ ]:
result.to_csv('/content/drive/MyDrive/Colab Notebooks/Data/Augmented_SM_Dataset.csv', index = False)